<a href="https://colab.research.google.com/github/vansh565/traffic-signals-detector-/blob/main/traffic_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive

 Classroom			 imgtopdf_1909242225053.pdf
'Colab Notebooks'		'Mindluster_Certificate 6 (1).pdf'
'Getting started.pdf'		'Mindluster_Certificate 6.pdf'
 gtsrb-german-traffic-sign.zip	 traffic_signs
 IMG_3332.heic			'Untitled folder'
 IMG_3332.HEIC			'VANSH 2.pdf'
 IMG_3333.jpeg			'Web Development Training - Certificate of Completion (1).pdf'


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
100% 611M/612M [00:03<00:00, 229MB/s]
100% 612M/612M [00:03<00:00, 199MB/s]


In [ ]:
!mv gtsrb-german-traffic-sign.zip /content/drive/My\ Drive/

In [ ]:
!unzip /content/drive/My\ Drive/gtsrb-german-traffic-sign.zip -d /content/drive/My\ Drive/traffic_signs

Archive:  /content/drive/My Drive/gtsrb-german-traffic-sign.zip
replace /content/drive/My Drive/traffic_signs/Meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
dataset_path = '/content/drive/My Drive/traffic_signs/'

In [ ]:
import pandas as pd
meta_data = pd.read_csv(dataset_path+'Meta.csv')
train_data = pd.read_csv(dataset_path+'Train.csv')
test_data = pd.read_csv(dataset_path+'Test.csv')
print(train_data.head())


In [ ]:
import cv2
import matplotlib.pyplot as plt
def visualize_image_with_bbox(data_row, img_folder):
    img_path = f"{img_folder}/{data_row['Path']}"
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x1, y1 = data_row['Roi.X1'], data_row['Roi.Y1']
    x2, y2 = data_row['Roi.X2'], data_row['Roi.Y2']
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    plt.imshow(img)
    plt.title(f"Class ID: {data_row['ClassId']}")
    plt.axis('off')
    plt.show()
visualize_image_with_bbox(train_data.iloc[0], dataset_path)


In [ ]:
!pip install keras

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
def crop_and_resize(data, img_folder, target_size=(64, 64)):
    images = []
    labels = []

    for _, row in data.iterrows():
        img_path = os.path.join(img_folder, row['Path'])
        img = cv2.imread(img_path)
        x1, y1, x2, y2 = row['Roi.X1'], row['Roi.Y1'], row['Roi.X2'], row['Roi.Y2']
        cropped_img = img[y1:y2, x1:x2]
        resized_img = cv2.resize(cropped_img, target_size)
        images.append(resized_img / 255.0)
        labels.append(row['ClassId'])

    return np.array(images), np.array(labels)

train_images, train_labels = crop_and_resize(train_data, dataset_path)
test_images, test_labels = crop_and_resize(test_data, dataset_path)


In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

train_labels_categorical = to_categorical(train_labels_encoded)
test_labels_categorical = to_categorical(test_labels_encoded)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_images, train_labels_categorical,
    validation_data=(test_images, test_labels_categorical),
    epochs=15,
    batch_size=32
)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels_categorical)
print(f"Test Accuracy: {test_acc:.2f}")


In [ ]:
!kaggle datasets download -d ganeshishwarpatil/gtsrblabel-names

In [ ]:
!mv gtsrblabel-names.zip /content/drive/My\ Drive/

In [ ]:
!unzip /content/drive/My\ Drive/gtsrblabel-names.zip -d /content/drive/My\ Drive/traffic_signs_labels

In [ ]:
label_path = '/content/drive/My Drive/traffic_signs_labels/'

In [ ]:
import pandas as pd
labels = pd.read_csv(label_path+'label_names.csv')
print(labels.head())

In [ ]:
def get_label(class_id):
    label = labels[labels['ClassId'] == class_id]['SignName'].values
    if len(label) > 0:
        return label[0]
    else:
        return "Unknown"

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
#apne google drive pe image upload karke, yahan uska path likh dena
image_path = '/content/drive/MyDrive/traffic_signs/Test/00001.png'
img = load_img(image_path, target_size=(64, 64))
img_array = img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.rectangle(img, (64, 64), (64, 64), (255, 0, 0), 2)
plt.imshow(img)
plt.title(f"Your Image")
plt.axis('off')
plt.show()
print(f"Predicted Class ID: {predicted_class[0]}")
label = get_label(predicted_class[0])
print(f"Predicted Sign: {label}")